<a href="https://colab.research.google.com/github/andrybrew/socialmediaanalytic/blob/master/008_text_mining_part_3_(textnetwork).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Mining Part 3

#Text Network Analysis

Though network analysis is most often used to describe relationships between people, some of the early pioneers of network analysis realized that it could also be applied to represent relationships between words. For example, one can represent a corpus of documents as a network where each node is a document, and the thickness or strength of the edges between them describes similarities between the words used in any two documents. Or, one can create a textnetwork where individual words are the nodes, and the edges between them describe the regularity with which they co-occur in documents.

There are multiple advantages to a network-based approach to automated text analysis. Just as clusters of social connections can help explain a range of outcomes, understanding patterns of connections between words helps identify their meaning in a more precise manner.Second, text networks can be built out of documents of any length, whereas topic models function poorly on short texts such as social media messages.

In this prcatice we will use NetworkX. NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. You can see the full documentation of NetworkX HERE

**Install & Import Libraries**

In [0]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import community
import seaborn as sns

## a.Network Construction

Here we construct a text network based on conversations about 'Berita Batubara' on Twitter.

**Import Data**

In [0]:
# Import Data
df_text = pd.read_csv('https://raw.githubusercontent.com/dianrdn/data/master/berita_batubara.csv', sep = ';')

# Show Data
df_text

**Construct an Edge List**

In [0]:
# Select Columns and Create an Edge List
df_networktext = df_text[['source', 'target']]

# Show Edgelist
df_networktext

In [0]:
# Drop Missing Target from the Edge List
df_networktext = df_networktext.mask(df_networktext.eq('None')).dropna()

# Show Edgelist
df_networktext

**Visualize Text Network**

In [0]:
# Contstruct a Network
G = nx.from_pandas_edgelist(df_networktext)

# Visualize the Network
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
nx.draw(G, with_labels=True, 
        node_color='skyblue', node_size=1200, 
        arrowstyle='->',arrowsize=20, edge_color='r',
        font_size=9,
        pos=nx.kamada_kawai_layout(G))

## b.Network Metrics and Measurement

**Centrality Measurement**

In graph theory and network analysis, indicators of centrality identify the most important vertices within a graph. Applications include identifying the most influential person(s) in a social network, key infrastructure nodes in the Internet or urban networks, and super-spreaders of disease. Centrality concepts were first developed in social network analysis, and many of the terms used to measure centrality reflect their sociological origin.

In [0]:
# Degree Centrality
degree = nx.degree_centrality(G)

# Sorted from the Highest
sorted(nx.degree(G), key=lambda x: x[1], reverse=True)[0:10]

In [0]:
# Betweenness Centrality
betweenness = nx.betweenness_centrality(G)

# Sorted from the Highest
sorted(nx.betweenness_centrality(G, normalized=True).items(), key=lambda x:x[1], reverse=True)[0:10]

In [0]:
# Closeness Centrality
closeness = nx.closeness_centrality(G)

# Sorted from the Highest
sorted(nx.closeness_centrality(G).items(), key=lambda x:x[1], reverse=True)[0:10]

In [0]:
# Eigenvector Centrality
eigenvector = nx.eigenvector_centrality(G)

# Sorted from the Highest
sorted(nx.eigenvector_centrality(G).items(), key=lambda x:x[1], reverse=True)[0:10]

Visualize Network based on Centrality Measurement

In [0]:
# Set Degree Dictionary
d = dict(degree)

# Visualize the Network
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
nx.draw(G, with_labels=True, 
        node_color='skyblue', nodelist=d.keys(),
        node_size=[v * 5000 for v in d.values()], 
        arrowstyle='->',arrowsize=20, edge_color='r',
        font_size=9,
        pos=nx.kamada_kawai_layout(G))

**Network Topology Measurement**

The configuration, or topology, of a network is key to determining its performance. Network topology is the way a network is arranged, including the physical or logical description of how links and nodes are set up to relate to each other.

In [0]:
# Show Number of Nodes
nx.number_of_nodes(G)

In [0]:
# Show Number of Edges
nx.number_of_edges(G)

In [0]:
# Show Graph Density
nx.density(G)

## c.Community Detection

Community detection is a fundamental problem in social network analysis consisting, roughly speaking, in dividing social actors (modelled as nodes in a social graph) with certain social connections (modelled as edges in the social graph) into densely knitted and highly related groups with each group well separated from different group members.

**Modularity Community**

In [0]:
# Import Module
from networkx.algorithms.community import greedy_modularity_communities

# Modularity Community Detection
communities_m = sorted(greedy_modularity_communities(G), key=len, reverse=True)
communities_m

In [0]:
# Set Node Community Function
def set_node_community(G, communities_m):
      '''Add community to node attributes'''
      for c, v_c in enumerate(communities_m):
        for v in v_c:
          # Add 1 to save 0 for external edges
          G.nodes[v]['community'] = c + 1      

In [0]:
# Set Colour Function
def get_color(i, r_off=1, g_off=1, b_off=1):
     '''Assign a color to a vertex.'''
     r0, g0, b0 = 0, 0, 0
     n = 16
     low, high = 0.1, 0.9
     span = high - low
     r = low + span * (((i + r_off) * 3) % n) / (n - 1)
     g = low + span * (((i + g_off) * 5) % n) / (n - 1)
     b = low + span * (((i + b_off) * 7) % n) / (n - 1)
     return (r, g, b) 

In [0]:
# Set Node Communities
community = set_node_community(G, communities_m)

# Set Node Color
node_color = [get_color(G.nodes[v]['community']) for v in G.nodes]

# Visualize the Network
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
nx.draw(G, with_labels=True, 
        node_color = node_color, node_size=1200, 
        arrowstyle='->',arrowsize=20, edge_color='r',
        font_size=9, map = plt.get_cmap('jet'),
        pos=nx.kamada_kawai_layout(G))